# BI Exam May 2025: COVID-19 Data

#### Created by Group 7 - Kamilla, Jeanette, Juvena

In [364]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
import sklearn.metrics as sm
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

# Set plot styles for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Data Preparation

### 1. Load the Data

Now that we have our tools ready, the next step is to load the COVID-19 dataset into Python so we can start analyzing it.

In this case, we’re working with a single dataset:

- **OWID COVID-19 Latest Data**: a CSV file that contains country-level information on cases, deaths, vaccinations, testing, and various socioeconomic indicators.

We'll use Pandas to read the CSV file and store it as a DataFrame. To make our code cleaner and reusable, we'll define a simple function that loads the data and performs some initial checks. This way, we can easily reload or replace the dataset if needed in future steps.

In [365]:
# File paths for the covid datasets. (dataset: last updated 2024-08-04)
dataset_covid = 'Data/owid-covid-latest.csv'

# Function to load the Excel files
def load_csv_to_dataframe(file_path):
    # Reads the Excel file and skips the first row if it contains a description or title
    df = pd.read_csv(file_path)
    return df

# Load datasets
print("..Loading COVID-19 dataset")
df_covid = load_csv_to_dataframe(dataset_covid)

..Loading COVID-19 dataset


### 2. Explore the Data

After loading the dataset, we want to explore it to understand what kind of information it contains and how it's structured.

To do this, we can use several helpful Pandas functions such as `shape`, `types`, `info()`, `head()`, `tail()`, `sample()`, `describe()` and `isnull().sum()`. These functions will give us insights into the number of rows and columns, the data types of each column, a summary of the data, and any missing values. 

This exploration is crucial as it helps us identify potential issues or areas that need further cleaning or transformation before we proceed with our analysis. 

In [366]:
# Check the shape of the DataFrame (rows, columns)
df_covid.shape

(247, 67)

In [367]:
# Display the types of attributes (colum names) in the DataFrame
df_covid.dtypes

iso_code                                    object
continent                                   object
location                                    object
last_updated_date                           object
total_cases                                float64
                                            ...   
population                                 float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 67, dtype: object

In [368]:
# Gives an overview of the DataFrame
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   iso_code                                    247 non-null    object 
 1   continent                                   235 non-null    object 
 2   location                                    247 non-null    object 
 3   last_updated_date                           247 non-null    object 
 4   total_cases                                 246 non-null    float64
 5   new_cases                                   242 non-null    float64
 6   new_cases_smoothed                          242 non-null    float64
 7   total_deaths                                246 non-null    float64
 8   new_deaths                                  243 non-null    float64
 9   new_deaths_smoothed                         243 non-null    float64
 10  total_cases_pe

In [369]:
# Display the first 5 rows of the DataFrame
df_covid.head()

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2024-08-04,235214.0,0.0,0.000,7998.0,0.0,0.0,...,NaN,37.746,0.50,64.83,0.511,4.112877e+07,NaN,NaN,NaN,NaN
1,OWID_AFR,NaN,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2024-08-04,335047.0,0.0,0.000,3605.0,0.0,0.0,...,51.2,NaN,2.89,78.57,0.795,2.842318e+06,NaN,NaN,NaN,NaN
3,DZA,Africa,Algeria,2024-08-04,272139.0,18.0,2.571,6881.0,0.0,0.0,...,30.4,83.741,1.90,76.88,0.748,4.490323e+07,NaN,NaN,NaN,NaN
4,ASM,Oceania,American Samoa,2024-08-04,8359.0,0.0,0.000,34.0,0.0,0.0,...,NaN,NaN,NaN,73.74,NaN,4.429500e+04,NaN,NaN,NaN,NaN


In [370]:
# Display the last 5 rows of the DataFrame
df_covid.tail()

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
242,WLF,Oceania,Wallis and Futuna,2024-08-04,3760.0,0.0,0.000,9.0,0.0,0.000,...,NaN,NaN,NaN,79.94,NaN,1.159600e+04,NaN,NaN,NaN,NaN
243,OWID_WRL,NaN,World,2024-08-14,775866783.0,47169.0,6738.429,7057132.0,815.0,116.429,...,34.635,60.130,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN,NaN
244,YEM,Asia,Yemen,2024-08-04,11945.0,0.0,0.000,2159.0,0.0,0.000,...,29.200,49.542,0.700,66.12,0.470,3.369661e+07,NaN,NaN,NaN,NaN
245,ZMB,Africa,Zambia,2024-08-04,349842.0,18.0,2.571,4077.0,0.0,0.000,...,24.700,13.938,2.000,63.89,0.584,2.001767e+07,NaN,NaN,NaN,NaN
246,ZWE,Africa,Zimbabwe,2024-08-04,266386.0,0.0,0.000,5740.0,0.0,0.000,...,30.700,36.791,1.700,61.49,0.571,1.632054e+07,NaN,NaN,NaN,NaN


In [371]:
# Display a random sample of 5 rows from the DataFrame
df_covid.sample(5)

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
15,AZE,Asia,Azerbaijan,2024-08-04,835757.0,63.0,9.0,10353.0,0.0,0.0,...,42.5,83.241,4.7,73.00,0.756,10358078.0,NaN,NaN,NaN,NaN
122,LSO,Africa,Lesotho,2024-08-04,36138.0,0.0,0.0,709.0,0.0,0.0,...,53.9,2.117,NaN,54.33,0.527,2305826.0,NaN,NaN,NaN,NaN
121,LBN,Asia,Lebanon,2024-08-04,1239904.0,0.0,0.0,10947.0,0.0,0.0,...,40.7,NaN,2.9,78.93,0.744,5489744.0,NaN,NaN,NaN,NaN
176,PCN,Oceania,Pitcairn,2024-08-04,4.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN
115,KIR,Oceania,Kiribati,2024-08-04,5085.0,0.0,0.0,24.0,0.0,0.0,...,58.9,NaN,1.9,68.37,0.630,131237.0,NaN,NaN,NaN,NaN


In [372]:
# Gives summary statistics for all numerical columns in the dataset
df_covid.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,2.460000e+02,242.000000,242.000000,2.460000e+02,243.000000,243.00000,246.000000,242.000000,242.000000,246.000000,...,145.000000,96.000000,173.000000,231.000000,190.000000,2.470000e+02,0.0,0.0,0.0,0.0
mean,1.336634e+07,885.607438,126.515355,1.198689e+05,14.032922,2.00472,203988.255797,22.204909,3.172136,1271.427736,...,32.909897,50.788844,3.097012,73.660866,0.722500,1.307656e+08,NaN,NaN,NaN,NaN
std,6.568130e+07,4854.786157,693.540908,5.747240e+05,92.179347,13.16853,200456.902140,82.962646,11.851812,1322.697453,...,13.621757,32.124848,2.555777,7.405725,0.149398,6.684333e+08,NaN,NaN,NaN,NaN
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,7.700000,1.188000,0.100000,53.280000,0.394000,4.700000e+01,NaN,NaN,NaN,NaN
25%,2.750950e+04,0.000000,0.000000,1.837500e+02,0.000000,0.00000,21257.766500,0.000000,0.000000,144.808250,...,22.600000,20.482000,1.300000,69.545000,0.603000,4.294955e+05,NaN,NaN,NaN,NaN
50%,2.320985e+05,0.000000,0.000000,2.205500e+03,0.000000,0.00000,135384.895000,0.000000,0.000000,877.689000,...,33.100000,49.690500,2.500000,75.050000,0.740000,5.970430e+06,NaN,NaN,NaN,NaN
75%,1.703974e+06,5.500000,0.785500,1.938850e+04,0.000000,0.00000,340625.300000,0.232000,0.033250,2032.222000,...,41.300000,82.686750,4.200000,79.285000,0.828750,2.895671e+07,NaN,NaN,NaN,NaN
max,7.758668e+08,47169.000000,6738.429000,7.057132e+06,815.000000,116.42900,763598.600000,672.437000,96.062000,6601.110000,...,78.100000,100.000000,13.800000,86.750000,0.957000,7.975105e+09,NaN,NaN,NaN,NaN


##### **2.1 Summary of exploring the data**

After exploring the dataframe, we found that it contains a large number of columns, many of which are not useful for our analysis or modeling goals. While some columns provide valuable information (like total cases, deaths, and vaccination rates), others are either redundant, mostly empty, or irrelevant.

This highlights the need for a thorough data cleaning step to remove unnecessary columns, handle missing values, and focus only on the most relevant features for our machine learning tasks.

### 3. Clean the Data

After loading and exploring the data, we need to clean it to ensure that our analysis is accurate and meaningful. Data cleaning involves several steps, including: checking for missing values, removing duplicates, and converting data types.

We start by doing a bit of cleaning of the big dataset, to remove rows and columns that are not relevant for our futher analysis and before we seperate the data into more specific datasets.

In [373]:
# Check for missing values in the DataFrame
df_covid.isnull().sum()

iso_code                                     0
continent                                   12
location                                     0
last_updated_date                            0
total_cases                                  1
                                          ... 
population                                   0
excess_mortality_cumulative_absolute       247
excess_mortality_cumulative                247
excess_mortality                           247
excess_mortality_cumulative_per_million    247
Length: 67, dtype: int64

The output above shows that many columns contain no values at all, so we will remove them to clean up the dataset.

In [374]:
# Before cleaning the data, we want to remove irrelevant OWID aggregate rows—such as those representing high-income, low-income, and other income groupings.
rows_to_remove = ["OWID_UMC", "OWID_WRL", "OWID_LMC", "OWID_LIC", "OWID_HIC"]
df_removed_rows = df_covid[~df_covid["iso_code"].isin(rows_to_remove)]

We are removing the 'low-income countries', 'lower-middle-income countries', 'upper-middle-income countries', 'high-income countries' and 'world' categories because they are too broad and lack specific country-level detail, making it difficult to draw meaningful conclusions without relying on assumptions.

In [375]:
# Checking if the above rows were removed
print(f"{df_covid.shape}")
print(f"Removed the {df_covid.shape[0] - df_removed_rows.shape[0]} OWID rows from the dataframe.")

(247, 67)
Removed the 5 OWID rows from the dataframe.


In [376]:
# We will drop all columns with no values at all like; excess_mortality_cumulative_absolute, excess_mortality_cumulative etc.
df_covid_removed_columns = df_removed_rows.dropna(axis=1, how='all')

In [377]:
# Check whether the columns were removed
print(f"COVID dataframe shape after removing columns: {df_covid_removed_columns.shape}")
print(f"Removed {df_covid.shape[1] - df_covid_removed_columns.shape[1]} columns from the dataframe.")

COVID dataframe shape after removing columns: (242, 52)
Removed 15 columns from the dataframe.


#### 3.1 Separating the data into different datasets

Now we separate the continent-level , age-level and health-level data into their own DataFrames so that we can clean and process them independently from the country-level data. This allows us to apply different cleaning steps based on the nature of the data, since data may have different structures or missing values compared to individual countries.

##### 3.1.1 Separating the continent-level data

In [378]:
# Function to filter the DataFrame based on a list of values
def filter_dataframe(df, values, filter_type='rows', row_filter_column=None):
    if filter_type == 'rows':
        if row_filter_column is None:
            raise ValueError("Must specify 'row_filter_column' when filtering rows.")
        return df[df[row_filter_column].isin(values)]
    elif filter_type == 'columns':
        # Keep only columns present in df and in values list (avoid key error)
        columns_to_keep = [col for col in values if col in df.columns]
        return df[columns_to_keep]
    else:
        raise ValueError("filter_type must be either 'rows' or 'columns'")

In [379]:
# Before removing the iso_code column, we want to secure the OWID fields for the continents since it could be relevant data to analyze.
rows_to_secure = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_continents = filter_dataframe(df_covid_removed_columns, rows_to_secure, filter_type='rows', row_filter_column='iso_code')

In [380]:
# Check if the rows were secured
df_continents

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population
1,OWID_AFR,NaN,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09
12,OWID_ASI,NaN,Asia,2024-08-14,301499099.0,4515.0,645.000,1637249.0,17.0,2.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.721383e+09
70,OWID_EUR,NaN,Europe,2024-08-14,252916868.0,39047.0,5578.143,2102483.0,162.0,23.143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.448078e+08
71,OWID_EUN,NaN,European Union (27),2024-08-14,185822587.0,25642.0,3663.143,1262988.0,150.0,21.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.501468e+08
161,OWID_NAM,NaN,North America,2024-08-04,124492666.0,454.0,64.857,1671178.0,619.0,88.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.003237e+08
166,OWID_OCE,NaN,Oceania,2024-08-11,15003352.0,1809.0,258.429,32918.0,16.0,2.286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.503886e+07
207,OWID_SAM,NaN,South America,2024-08-04,68809418.0,1308.0,186.857,1354187.0,1.0,0.143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.368167e+08


We now have a new seperate dataframe called `df_continent` that contains the continent-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [381]:
# Remove the columns that are irrelavnt since they contain no values at all like; population_density, median_age etc.
df_continents_romved_columns = df_continents.dropna(axis=1, how='all')
df_continents_romved_columns

,iso_code,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,population
1,OWID_AFR,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.000,9088.877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09
12,OWID_ASI,Asia,2024-08-14,301499099.0,4515.0,645.000,1637249.0,17.0,2.429,63948.200,...,258.0,193.0,192.83,78.14,73.33,38.45,0.0,10.0,0.0,4.721383e+09
70,OWID_EUR,Europe,2024-08-14,252916868.0,39047.0,5578.143,2102483.0,162.0,23.143,337990.340,...,64.0,17.0,187.88,70.33,66.29,49.02,0.0,2.0,0.0,7.448078e+08
71,OWID_EUN,European Union (27),2024-08-14,185822587.0,25642.0,3663.143,1262988.0,150.0,21.429,413754.220,...,64.0,17.0,211.29,75.11,72.86,62.74,0.0,2.0,0.0,4.501468e+08
161,OWID_NAM,North America,2024-08-04,124492666.0,454.0,64.857,1671178.0,619.0,88.429,205992.190,...,442.0,442.0,192.99,76.39,65.71,42.69,1.0,0.0,0.0,6.003237e+08
166,OWID_OCE,Oceania,2024-08-11,15003352.0,1809.0,258.429,32918.0,16.0,2.286,333039.800,...,1130.0,1130.0,196.18,64.30,62.33,56.40,25.0,0.0,0.0,4.503886e+07
207,OWID_SAM,South America,2024-08-04,68809418.0,1308.0,186.857,1354187.0,1.0,0.143,159838.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.368167e+08


The columns `new_vaccinations_smoothed_per_million`, `new_people_vaccinated_smoothed` and `new_people_vaccinated_smoothed_per_hundred` contain a lot of missing values so they are not necessary for our analysis and we will drop them from the `df_continents_cleaned` DataFrame. By removing them, we can simplify the DataFrame and focus on the most relevant features for our analysis.

In [382]:
# Check whether the columns were removed
print(f"df_continent shape after removing columns: {df_continents_romved_columns.shape}")
print(f"Removed {df_continents.shape[1] - df_continents_romved_columns.shape[1]} columns from the dataframe.")

df_continent shape after removing columns: (7, 29)
Removed 23 columns from the dataframe.


In [383]:
# Removing columns there are irrelevant 
df_continents_cleaned = df_continents_romved_columns.drop(['iso_code', 'new_vaccinations_smoothed_per_million', 'new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred'], axis=1)
df_continents_cleaned

,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,population
1,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.000,9088.877,0.025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09
12,Asia,2024-08-14,301499099.0,4515.0,645.000,1637249.0,17.0,2.429,63948.200,0.958,...,3.689439e+09,3.462095e+09,1.815177e+09,258.0,193.0,192.83,78.14,73.33,38.45,4.721383e+09
70,Europe,2024-08-14,252916868.0,39047.0,5578.143,2102483.0,162.0,23.143,337990.340,52.181,...,5.238143e+08,4.937513e+08,3.650999e+08,64.0,17.0,187.88,70.33,66.29,49.02,7.448078e+08
71,European Union (27),2024-08-14,185822587.0,25642.0,3663.143,1262988.0,150.0,21.429,413754.220,57.095,...,3.381196e+08,3.279674e+08,2.824388e+08,64.0,17.0,211.29,75.11,72.86,62.74,4.501468e+08
161,North America,2024-08-04,124492666.0,454.0,64.857,1671178.0,619.0,88.429,205992.190,0.751,...,4.585635e+08,3.944939e+08,2.562648e+08,442.0,442.0,192.99,76.39,65.71,42.69,6.003237e+08
166,Oceania,2024-08-11,15003352.0,1809.0,258.429,32918.0,16.0,2.286,333039.800,40.156,...,2.896050e+07,2.807290e+07,2.540095e+07,1130.0,1130.0,196.18,64.30,62.33,56.40,4.503886e+07
207,South America,2024-08-04,68809418.0,1308.0,186.857,1354187.0,1.0,0.143,159838.720,3.038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.368167e+08


In [384]:
# Check for duplicates in the DataFrame
df_continents_cleaned.duplicated().sum()

0

We still have some rows with missing values in the df_continents_cleaned DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

Since it is a small dataframe, we can't delete the row with missing values(NaN), since we will lose a lot of data. We choose to replace the missing values, even though it can have a high risk of giving wrong information and have a big impact. 

The first four we replaced using realistic data.
But since it took too much time, we’ll use the median to fill the remaining NaN values.

In [385]:
# method for replacing cell with a value
def replace_cell(df, row_filter, column, value):
    df.loc[row_filter, column] = value

# replace NaN for total_vaccinations for Africa. 1.084.500.000 is from Africa CDC, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'total_vaccinations', 1084500000)

# replace NaN for total_vaccinations for South America. 970.800.000 is from WTO-IMF COVID-19 Vaccine Trade Tracker, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'total_vaccinations', 970800000)

# replace NaN for people_vaccinated for Africa. 725.000.000 is from Africa CDC, which is offical and reliable
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'people_vaccinated', 725000000)

# replace NaN for people_vaccinated for South America. 351.310.000 is from Our World in Data which is offical and reliable used by WHO.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'people_vaccinated', 351310000)

# method for replacing cell with median 
def fill_na_with_median(df, column_name):
    median_value = df[column_name].median()
    print(f"Median of '{column_name}': {median_value:.2f}")
    df[column_name].fillna(median_value, inplace=True)


fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated')
fill_na_with_median(df_continents_cleaned, 'total_boosters')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations_smoothed')
fill_na_with_median(df_continents_cleaned, 'total_vaccinations_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'total_boosters_per_hundred')
df_continents_cleaned

Median of 'people_fully_vaccinated': 394493922.00
Median of 'total_boosters': 282438789.00
Median of 'new_vaccinations': 258.00
Median of 'new_vaccinations_smoothed': 193.00
Median of 'total_vaccinations_per_hundred': 192.99
Median of 'people_vaccinated_per_hundred': 75.11
Median of 'people_fully_vaccinated_per_hundred': 66.29
Median of 'total_boosters_per_hundred': 49.02


/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(median_value, inplace=True)


,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,population
1,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.000,9088.877,0.025,...,7.250000e+08,3.944939e+08,2.824388e+08,258.0,193.0,192.99,75.11,66.29,49.02,1.426737e+09
12,Asia,2024-08-14,301499099.0,4515.0,645.000,1637249.0,17.0,2.429,63948.200,0.958,...,3.689439e+09,3.462095e+09,1.815177e+09,258.0,193.0,192.83,78.14,73.33,38.45,4.721383e+09
70,Europe,2024-08-14,252916868.0,39047.0,5578.143,2102483.0,162.0,23.143,337990.340,52.181,...,5.238143e+08,4.937513e+08,3.650999e+08,64.0,17.0,187.88,70.33,66.29,49.02,7.448078e+08
71,European Union (27),2024-08-14,185822587.0,25642.0,3663.143,1262988.0,150.0,21.429,413754.220,57.095,...,3.381196e+08,3.279674e+08,2.824388e+08,64.0,17.0,211.29,75.11,72.86,62.74,4.501468e+08
161,North America,2024-08-04,124492666.0,454.0,64.857,1671178.0,619.0,88.429,205992.190,0.751,...,4.585635e+08,3.944939e+08,2.562648e+08,442.0,442.0,192.99,76.39,65.71,42.69,6.003237e+08
166,Oceania,2024-08-11,15003352.0,1809.0,258.429,32918.0,16.0,2.286,333039.800,40.156,...,2.896050e+07,2.807290e+07,2.540095e+07,1130.0,1130.0,196.18,64.30,62.33,56.40,4.503886e+07
207,South America,2024-08-04,68809418.0,1308.0,186.857,1354187.0,1.0,0.143,159838.720,3.038,...,3.513100e+08,3.944939e+08,2.824388e+08,258.0,193.0,192.99,75.11,66.29,49.02,4.368167e+08


##### 3.1.2 Separating the age-level data

In [386]:
# We are using the same function as above to seperate the age-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "median_age", "aged_65_older", "aged_70_older", "life_expectancy"]
df_age = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [387]:
# Check if the rows were secured
df_age

,continent,location,total_deaths_per_million,median_age,aged_65_older,aged_70_older,life_expectancy
0,Asia,Afghanistan,197.098,18.6,2.581,1.337,64.83
1,NaN,Africa,179.157,NaN,NaN,NaN,NaN
2,Europe,Albania,1274.926,38.0,13.188,8.643,78.57
3,Africa,Algeria,151.306,29.1,6.211,3.857,76.88
4,Oceania,American Samoa,702.988,NaN,NaN,NaN,73.74
...,...,...,...,...,...,...,...
241,Asia,Vietnam,433.444,32.6,7.150,4.718,75.40
242,Oceania,Wallis and Futuna,782.541,NaN,NaN,NaN,79.94
244,Asia,Yemen,56.484,20.3,2.922,1.583,66.12
245,Africa,Zambia,202.303,17.7,2.480,1.542,63.89


We now have a new seperate dataframe called `df_age` that contains the age-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [388]:
# Check for missing values in the DataFrame
df_age.isnull().sum()

continent                    7
location                     0
total_deaths_per_million     1
median_age                  45
aged_65_older               51
aged_70_older               47
life_expectancy             12
dtype: int64

In [389]:
# Drop all the rows with NaN values in the 'median_age' column
df_age_cleaned = df_age.dropna(subset=['median_age'])

In [390]:
# Do another check for missing values in the DataFrame
df_age_cleaned.isnull().sum()

continent                   0
location                    0
total_deaths_per_million    1
median_age                  0
aged_65_older               6
aged_70_older               2
life_expectancy             0
dtype: int64

There are still some missing values in the `df_age_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [391]:
# Fill NaN values with the median for the columns; total_deaths_per_million, aged_65_older and aged_70_older
fill_na_with_median(df_age_cleaned, "total_deaths_per_million")
fill_na_with_median(df_age_cleaned, "aged_65_older")
fill_na_with_median(df_age_cleaned, "aged_70_older")
df_age_cleaned

Median of 'total_deaths_per_million': 865.95
Median of 'aged_65_older': 6.29
Median of 'aged_70_older': 3.86


/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(median_value, inplace=True)
/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].fillna(median_value,

,continent,location,total_deaths_per_million,median_age,aged_65_older,aged_70_older,life_expectancy
0,Asia,Afghanistan,197.098,18.6,2.581,1.337,64.83
2,Europe,Albania,1274.926,38.0,13.188,8.643,78.57
3,Africa,Algeria,151.306,29.1,6.211,3.857,76.88
6,Africa,Angola,54.357,16.8,2.405,1.362,61.15
8,North America,Antigua and Barbuda,1572.412,32.1,6.933,4.631,77.02
...,...,...,...,...,...,...,...
240,South America,Venezuela,207.564,29.0,6.614,3.915,72.06
241,Asia,Vietnam,433.444,32.6,7.150,4.718,75.40
244,Asia,Yemen,56.484,20.3,2.922,1.583,66.12
245,Africa,Zambia,202.303,17.7,2.480,1.542,63.89


In [392]:
# Check for duplicates in the DataFrame
df_age_cleaned.duplicated().sum()

0

##### 3.1.3 Separating the health-level data

In [393]:
# We are using the same function as above to seperate the health-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "life_expectancy"]
df_health = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [394]:
# Check if the rows were secured
df_health

,continent,location,total_deaths_per_million,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,life_expectancy
0,Asia,Afghanistan,197.098,597.029,9.59,NaN,NaN,64.83
1,NaN,Africa,179.157,NaN,NaN,NaN,NaN,NaN
2,Europe,Albania,1274.926,304.195,10.08,7.1,51.2,78.57
3,Africa,Algeria,151.306,278.364,6.73,0.7,30.4,76.88
4,Oceania,American Samoa,702.988,283.750,NaN,NaN,NaN,73.74
...,...,...,...,...,...,...,...,...
241,Asia,Vietnam,433.444,245.465,6.00,1.0,45.9,75.40
242,Oceania,Wallis and Futuna,782.541,NaN,NaN,NaN,NaN,79.94
244,Asia,Yemen,56.484,495.003,5.35,7.6,29.2,66.12
245,Africa,Zambia,202.303,234.499,3.94,3.1,24.7,63.89


We now have a new seperate dataframe called `df_health` that contains age-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [395]:
# Check for missing values in the DataFrame
df_health.isnull().sum()

continent                    7
location                     0
total_deaths_per_million     1
cardiovasc_death_rate       48
diabetes_prevalence         37
female_smokers              96
male_smokers                98
life_expectancy             12
dtype: int64

In [396]:
# Drop all the rows with NaN values in the 'female_smokers' column
df_health_cleaned = df_health.dropna(subset=['female_smokers'])

In [397]:
# Do another check for missing values in the DataFrame
df_health_cleaned.isnull().sum()

continent                   0
location                    0
total_deaths_per_million    0
cardiovasc_death_rate       2
diabetes_prevalence         0
female_smokers              0
male_smokers                2
life_expectancy             0
dtype: int64

There are still some missing values in the `df_health_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [398]:
# Fill NaN values with the median for the columns; cardiovasc_death_rate and male_smokers
fill_na_with_median(df_health_cleaned, "cardiovasc_death_rate")
fill_na_with_median(df_health_cleaned, "male_smokers")
df_health_cleaned

Median of 'cardiovasc_death_rate': 239.27
Median of 'male_smokers': 32.25


/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(median_value, inplace=True)
/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].fillna(median_value,

,continent,location,total_deaths_per_million,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,life_expectancy
2,Europe,Albania,1274.926,304.195,10.08,7.1,51.2,78.57
3,Africa,Algeria,151.306,278.364,6.73,0.7,30.4,76.88
5,Europe,Andorra,1994.431,109.135,7.97,29.0,37.8,83.73
9,South America,Argentina,2877.538,191.032,5.50,16.2,27.7,76.67
10,Asia,Armenia,3046.635,341.010,7.11,1.5,52.1,75.09
...,...,...,...,...,...,...,...,...
238,Oceania,Vanuatu,44.720,546.300,12.02,2.8,34.5,70.47
241,Asia,Vietnam,433.444,245.465,6.00,1.0,45.9,75.40
244,Asia,Yemen,56.484,495.003,5.35,7.6,29.2,66.12
245,Africa,Zambia,202.303,234.499,3.94,3.1,24.7,63.89


In [399]:
# Check for duplicates in the DataFrame
df_health_cleaned.duplicated().sum()

0

#### 3.2 Futher cleaning of the country-level data 

We have selected a subset of columns that we consider relevant for our analysis. This subset includes columns that provide information on total cases, deaths and population. By focusing on these columns, we can simplify our analysis and make it easier to draw meaningful conclusions.

In [400]:
# We make a new dataframe with the columns we want to keep for future analysis.
columns_we_want_to_keep = [
    "iso_code", "continent", "location", "total_cases", "total_deaths",
    "total_cases_per_million", "total_deaths_per_million",
    "life_expectancy", "population"]

# Removes all other columns
df_covid = df_covid_removed_columns[columns_we_want_to_keep]

In [401]:
# Check if the columns were removed
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242 entries, 0 to 246
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   iso_code                  242 non-null    object 
 1   continent                 235 non-null    object 
 2   location                  242 non-null    object 
 3   total_cases               241 non-null    float64
 4   total_deaths              241 non-null    float64
 5   total_cases_per_million   241 non-null    float64
 6   total_deaths_per_million  241 non-null    float64
 7   life_expectancy           230 non-null    float64
 8   population                242 non-null    float64
dtypes: float64(6), object(3)
memory usage: 18.9+ KB


We then load another dataset so we can add data about the Human Development Index (HDI) for each country. The HDI is a composite index of life expectancy, education, and per capita income indicators, which are used to rank countries into four tiers of human development. This additional information will help us better understand the relationship between COVID-19 and various socioeconomic factors.

In [402]:
# We load the new dataset
hdi = pd.read_csv('Data/human-development-index.csv')

Because we can then add a column with the HDI data for 2021 matching the countries in the covid dataset, because we only need data from the last year.

In [403]:
# Filter HDI for 2021 only
hdi_2021 = hdi[hdi['Year'] == 2021]

# Merge using 'location' from df_covid and 'Entity' from hdi
df_merged = df_covid.merge(
    hdi_2021[['Entity', 'Human Development Index']], 
    left_on='location', 
    right_on='Entity', 
    how='left'
)

# Drop the extra 'Entity' column after merge, since we don't need it
df_merged = df_merged.drop(columns=['Entity'])

# Rename the column in the merged dataframe
df_merged = df_merged.rename(columns={'Human Development Index': 'human_development_index'})

In [404]:
# Check how the dataset look and how we should proceed
df_merged

,iso_code,continent,location,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million,life_expectancy,population,human_development_index
0,AFG,Asia,Afghanistan,235214.0,7998.0,5796.468,197.098,64.83,4.112877e+07,0.478
1,OWID_AFR,NaN,Africa,13145380.0,259117.0,9088.877,179.157,NaN,1.426737e+09,NaN
2,ALB,Europe,Albania,335047.0,3605.0,118491.020,1274.926,78.57,2.842318e+06,0.796
3,DZA,Africa,Algeria,272139.0,6881.0,5984.050,151.306,76.88,4.490323e+07,0.745
4,ASM,Oceania,American Samoa,8359.0,34.0,172831.600,702.988,73.74,4.429500e+04,NaN
...,...,...,...,...,...,...,...,...,...,...
237,VNM,Asia,Vietnam,11624000.0,43206.0,116612.400,433.444,75.40,9.818686e+07,0.703
238,WLF,Oceania,Wallis and Futuna,3760.0,9.0,326928.100,782.541,79.94,1.159600e+04,NaN
239,YEM,Asia,Yemen,11945.0,2159.0,312.509,56.484,66.12,3.369661e+07,0.455
240,ZMB,Africa,Zambia,349842.0,4077.0,17359.357,202.303,63.89,2.001767e+07,0.565


In [405]:
# Shape of the dataframe after some cleaning
print(f"COVID dataframe shape after removing both some columns and rows: {df_merged.shape}")

COVID dataframe shape after removing both some columns and rows: (242, 10)


We are isolating the remaining rows in the df_covid DataFrame to ensure it contains only country-level data. This allows us to clean the dataset and retain only the features that are most relevant for our analysis.

In [406]:
# Since we seperated the OWID continent fields into it's own dataframe earlier, we now have to remove them again for the df_covid dataframe.
rows_to_remove = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_covid_removed_rows = df_merged[~df_merged['iso_code'].isin(rows_to_remove)]
df_covid_cleaned = df_covid_removed_rows.dropna(subset=['iso_code'])
df_covid_cleaned = df_covid_cleaned.drop(columns=['iso_code'])
df_covid_cleaned        

,continent,location,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million,life_expectancy,population,human_development_index
0,Asia,Afghanistan,235214.0,7998.0,5796.468,197.098,64.83,41128772.0,0.478
2,Europe,Albania,335047.0,3605.0,118491.020,1274.926,78.57,2842318.0,0.796
3,Africa,Algeria,272139.0,6881.0,5984.050,151.306,76.88,44903228.0,0.745
4,Oceania,American Samoa,8359.0,34.0,172831.600,702.988,73.74,44295.0,NaN
5,Europe,Andorra,48015.0,159.0,602280.440,1994.431,83.73,79843.0,0.858
...,...,...,...,...,...,...,...,...,...
237,Asia,Vietnam,11624000.0,43206.0,116612.400,433.444,75.40,98186856.0,0.703
238,Oceania,Wallis and Futuna,3760.0,9.0,326928.100,782.541,79.94,11596.0,NaN
239,Asia,Yemen,11945.0,2159.0,312.509,56.484,66.12,33696612.0,0.455
240,Africa,Zambia,349842.0,4077.0,17359.357,202.303,63.89,20017670.0,0.565


In [407]:
# Check whether the rows were removed
print(f"COVID dataframe shape after removing rows: {df_covid_cleaned.shape}")
print(f"Removed {df_merged.shape[0] - df_covid_cleaned.shape[0]} rows from the dataframe.")
print(f"Removed {df_merged.shape[1] - df_covid_cleaned.shape[1]} column from the dataframe.")

COVID dataframe shape after removing rows: (235, 9)
Removed 7 rows from the dataframe.
Removed 1 column from the dataframe.


In [408]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

continent                    0
location                     0
total_cases                  1
total_deaths                 1
total_cases_per_million      1
total_deaths_per_million     1
life_expectancy              5
population                   0
human_development_index     44
dtype: int64

We can see a lot of missing values for the human_development_index column, so we will impute them with the HDI from the sovereign countries they belong too. This step is important because missing values can lead to biased results or errors in our models.

In [409]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

['American Samoa' 'Anguilla' 'Aruba' 'Bermuda'
 'Bonaire Sint Eustatius and Saba' 'British Virgin Islands'
 'Cayman Islands' 'Cook Islands' 'Curacao' 'Falkland Islands'
 'Faroe Islands' 'French Guiana' 'French Polynesia' 'Gibraltar'
 'Greenland' 'Guadeloupe' 'Guam' 'Guernsey' 'Isle of Man' 'Jersey'
 'Kosovo' 'Martinique' 'Mayotte' 'Monaco' 'Montserrat' 'Nauru'
 'New Caledonia' 'Niue' 'North Korea' 'Northern Mariana Islands'
 'Pitcairn' 'Puerto Rico' 'Reunion' 'Saint Barthelemy' 'Saint Helena'
 'Saint Martin (French part)' 'Saint Pierre and Miquelon'
 'Sint Maarten (Dutch part)' 'Somalia' 'Tokelau'
 'Turks and Caicos Islands' 'United States Virgin Islands' 'Vatican'
 'Wallis and Futuna']


In [410]:
territory_to_country = {
    'American Samoa': 'United States',
    'Anguilla': 'United Kingdom',
    'Aruba': 'Netherlands',
    'Bermuda': 'United Kingdom',
    'Bonaire Sint Eustatius and Saba': 'Netherlands',
    'British Virgin Islands': 'United Kingdom',
    'Cayman Islands': 'United Kingdom',
    'Cook Islands': 'New Zealand',
    'Curacao': 'Netherlands',
    'Falkland Islands': 'United Kingdom',
    'Faroe Islands': 'Denmark',
    'French Guiana': 'France',
    'French Polynesia': 'France',
    'Gibraltar': 'United Kingdom',
    'Greenland': 'Denmark',
    'Guadeloupe': 'France',
    'Guam': 'United States',
    'Guernsey': 'United Kingdom',
    'Isle of Man': 'United Kingdom',
    'Jersey': 'United Kingdom',
    'Kosovo': 'Serbia',  # or leave as is if Kosovo has its own HDI
    'Martinique': 'France',
    'Mayotte': 'France',
    'Monaco': 'France',
    'Montserrat': 'United Kingdom',
    'Nauru': 'Nauru',
    'New Caledonia': 'France',
    'Niue': 'New Zealand',
    'North Korea': 'North Korea',
    'Northern Mariana Islands': 'United States',
    'Pitcairn': 'United Kingdom',
    'Puerto Rico': 'United States',
    'Reunion': 'France',
    'Saint Barthelemy': 'France',
    'Saint Helena': 'United Kingdom',
    'Saint Martin (French part)': 'France',
    'Saint Pierre and Miquelon': 'France',
    'Sint Maarten (Dutch part)': 'Netherlands',
    'Somalia': 'Somalia',
    'Tokelau': 'New Zealand',
    'Turks and Caicos Islands': 'United Kingdom',
    'United States Virgin Islands': 'United States',
    'Vatican': 'Italy',
    'Wallis and Futuna': 'France'
}

In [411]:
# Map the territory to its sovereign country:
df_covid_cleaned['hdi_source_country'] = df_covid_cleaned['location'].map(territory_to_country)

In [412]:
# Create a lookup for HDI values of sovereign countries
hdi_lookup = df_covid_cleaned.set_index('location')['human_development_index'].to_dict()

In [413]:
# Fill missing HDI values with the sovereign country’s HDI
df_covid_cleaned['human_development_index'] = df_covid_cleaned.apply(
    lambda row: hdi_lookup.get(row['hdi_source_country'], row['human_development_index']) 
    if pd.isna(row['human_development_index']) else row['human_development_index'],
    axis=1
)

df_covid_cleaned.drop(columns=['hdi_source_country'], inplace=True)

In [414]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

continent                   0
location                    0
total_cases                 1
total_deaths                1
total_cases_per_million     1
total_deaths_per_million    1
life_expectancy             5
population                  0
human_development_index     3
dtype: int64

In [415]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

['Nauru' 'North Korea' 'Somalia']


We have found data for the human_development_index for 2021 for Nauru and Somalia on the https://hdr.undp.org/ website. We will use this data to fill in the missing values for these two countries in our dataset. We weren't able to find data on the human_development_index for North Korea, so we will impute it with the median value of the HDI for the other countries in the dataset. 

We will also impute the missing values for the columns total_cases, total_deaths, total_cases_per_million, total_deaths_per_million and life_expectancy with the median value.

In [422]:
# Replace missing HDI values for Nauru and Somalia and impute North Korea with median value
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Nauru', 'human_development_index', 0.692)
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Somalia', 'human_development_index', 0.385)
fill_na_with_median(df_covid_cleaned, "human_development_index")
fill_na_with_median(df_covid_cleaned, "total_cases")
fill_na_with_median(df_covid_cleaned, "total_deaths")
fill_na_with_median(df_covid_cleaned, "total_cases_per_million")
fill_na_with_median(df_covid_cleaned, "total_deaths_per_million")
fill_na_with_median(df_covid_cleaned, "life_expectancy")
df_covid_cleaned


Median of 'human_development_index': 0.77
Median of 'total_cases': 189095.00
Median of 'total_deaths': 1736.50
Median of 'total_cases_per_million': 135384.90
Median of 'total_deaths_per_million': 858.50
Median of 'life_expectancy': 75.05


/var/folders/2m/35grmzwx11n7k6l_7l4ycfhm0000gp/T/ipykernel_95505/3133756775.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(median_value, inplace=True)


,continent,location,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million,life_expectancy,population,human_development_index
0,Asia,Afghanistan,235214.0,7998.0,5796.468,197.098,64.83,41128772.0,0.478
2,Europe,Albania,335047.0,3605.0,118491.020,1274.926,78.57,2842318.0,0.796
3,Africa,Algeria,272139.0,6881.0,5984.050,151.306,76.88,44903228.0,0.745
4,Oceania,American Samoa,8359.0,34.0,172831.600,702.988,73.74,44295.0,0.921
5,Europe,Andorra,48015.0,159.0,602280.440,1994.431,83.73,79843.0,0.858
...,...,...,...,...,...,...,...,...,...
237,Asia,Vietnam,11624000.0,43206.0,116612.400,433.444,75.40,98186856.0,0.703
238,Oceania,Wallis and Futuna,3760.0,9.0,326928.100,782.541,79.94,11596.0,0.903
239,Asia,Yemen,11945.0,2159.0,312.509,56.484,66.12,33696612.0,0.455
240,Africa,Zambia,349842.0,4077.0,17359.357,202.303,63.89,20017670.0,0.565


In [423]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

continent                   0
location                    0
total_cases                 0
total_deaths                0
total_cases_per_million     0
total_deaths_per_million    0
life_expectancy             0
population                  0
human_development_index     0
dtype: int64

In [416]:
# Check for duplicates in the DataFrame
df_covid_cleaned.duplicated().sum()

0